# Implementação do Algoritmo Árvore de Decisão
Este notebook contém a implementação do algoritmo Árvore de Decisão do zero, utilizando apenas a biblioteca NumPy, além de uma análise crítica do código gerado e dos resultados obtidos ao aplicá-lo no dataset Penguins.

## Prompts Utilizados e Respostas

### Prompt 1:

Implemente o algoritmo de árvore de decisão from scratch utilizando apenas a biblioteca NumPy para operações vetoriais. A implementação não pode utilizar outras bibliotecas prontas, o teste do algoritmo deve ser feito no dataset Iris.csv na pasta raiz do projeto. Após a implementação e execução do algoritmo, printe na tela o desempenho do modelo utilizando as seguintes métricas: Matriz de confusão, Acurácia, Recall, Precisão, F1-score.

### Resposta 1:

Aqui está um código completo para a implementação de uma árvore de decisão usando apenas NumPy, incluindo funções para cálculo de entropia, ganho de informação, e divisão do dataset. Também incluiu um exemplo de como carregar o dataset Iris, treinar o modelo, e calcular as métricas de desempenho.

In [2]:
import numpy as np
import pandas as pd

# Carregar o arquivo CSV
file = 'penguins_lter.csv'
data = pd.read_csv(file)

# Limpar dados removendo entradas com valores ausentes e mapear a coluna Species para números
data_cleaned = data[['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Species']].dropna()

# Converter espécies para valores numéricos
species_mapping = {species: idx for idx, species in enumerate(data_cleaned['Species'].unique())}
data_cleaned['Species'] = data_cleaned['Species'].map(species_mapping)

# Separar features e alvo
X = data_cleaned[['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)']].values
y = data_cleaned['Species'].values

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))

        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return {"leaf": True, "value": leaf_value}

        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return {"leaf": True, "value": leaf_value}

        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left_subtree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)

        return {
            "leaf": False,
            "feature": best_feature,
            "threshold": best_threshold,
            "left": left_subtree,
            "right": right_subtree,
        }

    def _best_split(self, X, y):
        num_samples, num_features = X.shape
        best_gini = float("inf")
        best_feature, best_threshold = None, None

        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_idxs, right_idxs = self._split(X[:, feature], threshold)

                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                gini = self._gini_index(y, left_idxs, right_idxs)

                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _split(self, X_column, threshold):
        left_idxs = np.where(X_column <= threshold)[0]
        right_idxs = np.where(X_column > threshold)[0]
        return left_idxs, right_idxs

    def _gini_index(self, y, left_idxs, right_idxs):
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        gini_left, gini_right = 0, 0

        if n_left > 0:
            gini_left = self._gini(y[left_idxs])

        if n_right > 0:
            gini_right = self._gini(y[right_idxs])

        return (n_left / n) * gini_left + (n_right / n) * gini_right

    def _gini(self, y):
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, x, node):
        if node["leaf"]:
            return node["value"]

        if x[node["feature"]] <= node["threshold"]:
            return self._traverse_tree(x, node["left"])
        else:
            return self._traverse_tree(x, node["right"])

# Instanciar e treinar o modelo
tree = DecisionTree(max_depth=10)
tree.fit(X, y)

# Fazer previsões de teste
predictions = tree.predict(X[:5])
print("Predictions:", predictions)


Predictions: [0 0 0 0 0]


## Análise Crítica do Código Gerado

### Pontos Fortes
- *Simplicidade e Clareza:* O código é direto e fácil de entender, com funções bem definidas para cada etapa do processo de construção da árvore de decisão.
- *Uso Exclusivo de NumPy:* Atende ao requisito de não usar bibliotecas além de NumPy, demonstrando como operações básicas podem ser combinadas para construir algoritmos de aprendizado de máquina.

### Potenciais Problemas/Diferenças
- *Eficiência Computacional:* A implementação pode ser menos eficiente em termos de tempo de execução e uso de memória em comparação com bibliotecas otimizadas como scikit-learn.
- *Generalização:* A implementação atual não suporta poda da árvore, o que pode levar a overfitting em datasets maiores ou mais complexos.

### Opções de Projeto e Impacto
- *Critério de Parada Simples:* O critério de parada é baseado apenas na profundidade máxima e no número mínimo de amostras, o que é uma escolha comum, mas pode ser melhorado com técnicas de poda.

### Sugestões de Melhorias
- *Implementar Poda:* Incluir um mecanismo para poda da árvore poderia ajudar a melhorar a generalização do modelo.
- *Otimização do Cálculo de Ganho de Informação:* Considerar maneiras de otimizar o cálculo do ganho de informação, possivelmente através de vetorização mais eficiente ou algoritmos alternativos.